# Chapter 4: Linear methods for classifiction

## 4.2: Linear regression of an Indicator Matrix

In [1]:
# Import dependent packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import scale
import scipy as sp
from scipy import linalg
plt.style.use('seaborn')
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn import metrics 
from sklearn.linear_model import LogisticRegression


In [2]:
### Function takes two feature transform methods: normalization and MiniMaxScale
def feature_prerpocess(data, form = 'normal'):
    if form == "normal":
        new_data = scale(data)
    elif form == "MinMax":
        new_data = minmax_scale(data)
    return pd.DataFrame(new_data)

As name suggests we construct a response indaicator matrix instead of a vector in a linear regression model. Each of the response categories are coded via an indicator variable. Our model is of the form:

\begin{equation}
Y = XB
\end{equation}
 where $Y$ is $n \times k$ response matrix, $X$ is $n \times (k+1)$ matrix of observed data, $B$ is $(k+1) \times k$ matrix of coefficients and $k$ is a number of classes/categories.<br>
As in case of OLS,

\begin{equation}
\hat{B} =(\mathrm{X^TX})^{-1}X^Ty 
\end{equation}

A new observation is classified as follows:<br>
1. estimate the fitted output $\hat{y}=(1,x_T)\hat{B}$, a $k$ vector
2. classify: 
\begin{equation}
\hat{G(x)} =argmax_{k \subseteq G}\hat{y_k}(x)
\end{equation}

We will use "Audit" dataset, it can be downloaded:
* <a href="https://archive.ics.uci.edu/ml/datasets/Audit+Data#">Audit Dataset
    
The dataset contains 777 different firms that are collected from six distinct sectors. The information about the sectors and the counts of firms are listed respectively as Irrigation (114), Public Health (77), Buildings and Roads (82), Forest (70), Corporate (47), Animal Husbandry (95), Communication (1), Electrical (4), Land (5), Science and Technology (3), Tourism (1), Fisheries (41), Industries (37), Agriculture (200). Many risk factors are examined from various areas like past records of audit office, audit-paras, environmental conditions reports, firm reputation summary, on-going issues report, profit-value records, loss-value records, follow-up reports etc. Thus we buikd a classification model that can predict the fraudulent firm on the basis the present and historical risk factors.

In [3]:
#Download data 
Audit = pd.read_csv('audit_risk.csv', sep=',', encoding= 'unicode_escape')
Audit.describe()

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
count,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,...,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.0,776.000000,776.000000
mean,20.184536,2.450194,0.351289,1.351029,10.799988,0.313144,6.334008,13.218481,5.067655,0.223711,...,0.519072,0.104381,0.216753,0.053608,2.702577,17.680612,0.572680,0.5,7.168158,0.393041
std,24.319017,5.678870,0.174055,3.440447,50.083624,0.169804,30.072845,51.312829,0.264449,0.080352,...,0.290312,0.531031,0.067987,0.305835,0.858923,54.740244,0.444581,0.0,38.667494,0.488741
min,1.850000,0.000000,0.200000,0.000000,0.000000,0.200000,0.000000,0.000000,5.000000,0.200000,...,0.400000,0.000000,0.200000,0.000000,2.000000,1.400000,0.400000,0.5,0.280000,0.000000
25%,2.370000,0.210000,0.200000,0.042000,0.000000,0.200000,0.000000,0.537500,5.000000,0.200000,...,0.400000,0.000000,0.200000,0.000000,2.000000,1.583500,0.400000,0.5,0.316700,0.000000
50%,3.890000,0.875000,0.200000,0.175000,0.405000,0.200000,0.081000,1.370000,5.000000,0.200000,...,0.400000,0.000000,0.200000,0.000000,2.400000,2.214000,0.400000,0.5,0.555600,0.000000
75%,55.570000,2.480000,0.600000,1.488000,4.160000,0.400000,1.840500,7.707500,5.000000,0.200000,...,0.400000,0.000000,0.200000,0.000000,3.250000,10.663500,0.400000,0.5,3.249900,1.000000
max,59.850000,85.000000,0.600000,51.000000,1264.630000,0.600000,758.778000,1268.910000,9.000000,0.600000,...,2.400000,9.000000,0.600000,5.400000,5.200000,801.262000,5.800000,0.5,961.514400,1.000000


In [4]:
Audit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sector_score    776 non-null    float64
 1   LOCATION_ID     776 non-null    object 
 2   PARA_A          776 non-null    float64
 3   Score_A         776 non-null    float64
 4   Risk_A          776 non-null    float64
 5   PARA_B          776 non-null    float64
 6   Score_B         776 non-null    float64
 7   Risk_B          776 non-null    float64
 8   TOTAL           776 non-null    float64
 9   numbers         776 non-null    float64
 10  Score_B.1       776 non-null    float64
 11  Risk_C          776 non-null    float64
 12  Money_Value     775 non-null    float64
 13  Score_MV        776 non-null    float64
 14  Risk_D          776 non-null    float64
 15  District_Loss   776 non-null    int64  
 16  PROB            776 non-null    float64
 17  RiSk_E          776 non-null    flo

As we can see there is no missing values in the data. The dataset have 26 feature variables and 1 cateogrical response variable. We drop "LOCATION_ID" feature as it has no a predictive power.

In [5]:
#Feature matrix
X_matrix = Audit[Audit.columns[~Audit.columns.isin(["LOCATION_ID", 'Risk'])]]
X_matrix = X_matrix.fillna(method ='pad') 

The first column of 1's in the design matrix allows estimation of the y-intercept, so we add a new column to our data matrix $X$:

In [6]:
X_matrix.insert(0,'intercept',np.ones((len(X_matrix),)).T)

In [7]:
# Response indicator matrix
Y = Audit[["Risk"]]
Y_var = Audit[["Risk"]]
Y = Y.astype('str') 
Y_matrix = pd.get_dummies(Y)

Below we compute OLS estimator for $B$ matrix of paramaters:

In [8]:
# Paramater estimation:
beta = (linalg.inv(X_matrix.T@X_matrix))@X_matrix.T@Y_matrix
beta.columns  = ['Not fraudulent firm', 'fraudulent firm']
beta

,Not fraudulent firm,fraudulent firm
0,1.040734,-0.714099
1,0.001004,-0.001004
2,0.061286,-0.061286
3,0.933510,-1.036901
4,0.020361,-0.020361
5,-0.084466,0.084466
6,4.622363,-4.185401
7,0.261808,-0.261808
8,0.012569,-0.012569
9,0.027185,-0.027185


In [9]:
# Estimation of fitted values
D = X_matrix@beta.to_numpy()
Y_fitted = pd.DataFrame(np.where(D.iloc[:,0]<D.iloc[:,1], 1, 0))

Error rate (ERR) is calculated as the number of all incorrect predictions divided by the total number of the dataset:

\begin{equation}
\frac{FP+FN}{P+N}
\end{equation}

Error rate is calculated as the total number of two incorrect predictions (FN + FP) divided by the total number of a dataset (P + N).

In [10]:
# Calculate error rate
Error_rate = np.sum(np.abs(Y_fitted.iloc[:,0]-Y_var.iloc[:,0]))/len(Y_fitted)
Error_rate

0.20231958762886598

## 4.3: Discriminant Analysis

###  Linear Discriminant Analysis

Posterior probability of $P(G|X)$ is given by Bayes theorem:

\begin{equation}
P(G=k|X=x) = \frac{f_k(x)\pi_k}{\sum_{l=1}^{K} f_l(x)\pi_l}
\end{equation}

where, $f_k(x)$ is the conditional density of $X$ in class $G=k$, and $\pi_k$ is the prior probability  of class $k$, with $\sum_{l=1}^{K}\pi_l=1$ <br>
In LDA we assume:
1. each class density is Normally distributed,
2. the classes have a common variance.

If we have p variables then each class has a multivariate Normal distribution:

\begin{equation}
f_k(x) = \frac{1}{(2\pi)^{p/2}|\Sigma|^{1/2}}e^{-\frac{1}{2}(x-\mu_k)^T\Sigma_k^{-1}(x-\mu_k)}
\end{equation}

As a decision boundary we use log-ratio, which is linear in $x$ (that's why it is called Linear Discriminant Analysis):

\begin{equation}
log\frac{P(G=k|X=x)}{P(G=l|X=x)} = log\frac{f_k(x)}{f_l(x)}+log\frac{\pi_k}{\pi_l}=log\frac{\pi_k}{\pi_l}-\frac{1}{2}(\mu_k-\mu_l)^T\Sigma^{-1}(\mu_k-\mu_l)+x^T\Sigma^{-1}(\mu_k-\mu_l)
\end{equation}

or

\begin{equation}
\delta_k(x)=x^T\Sigma^{-1}\mu_k-\frac{1}{2}(\mu_k)^T\Sigma^{-1}(\mu_k)+log\pi_k
\end{equation}

We do not know the parameters of the Normal distributions, and we will estimate them using our training data:
- $\hat{\pi_k}=\frac{N_k}{N}$, where $N_k$ is the number of <em>class-k</em> observations;
- $\hat{\mu_k}=\sum_{g_i=k}^{}/N_k$;
- $\hat{\Sigma}=\frac{\sum_{k=k}^{K}\sum_{g_i=k}^{}(x_i-\hat{\mu_k})^T}{(N-K)}$

In [11]:
# Drop intercept from the data matrix
X_matrix = X_matrix.drop(['intercept'], axis=1)
X_matrix = feature_prerpocess(X_matrix, form = 'normal')

In [12]:
## Estimation of the paramaters of Normal distributions
# 1) Prior probabilities
prior_prob = pd.DataFrame({"fraudulent firm": np.sum(Y_var)/len(Y_var), "Not fraudulent firm":(len(Y_var)-np.sum(Y_var))/len(Y_var)})

#2) means
fraudulent_firm = pd.concat([X_matrix,Y_var],axis=1)[pd.concat([X_matrix,Y_var],axis=1)["Risk"]==1]
Not_fraudulent_firm = pd.concat([X_matrix,Y_var],axis=1)[pd.concat([X_matrix,Y_var],axis=1)["Risk"]==0]
means_class = pd.DataFrame({"fraudulent firm": np.mean(fraudulent_firm.iloc[:,:-1]), "Not fraudulent firm":np.mean(Not_fraudulent_firm.iloc[:,:-1])})

#3) Covariance matrix
covariance_fraudulent_firm = np.subtract(fraudulent_firm.iloc[:,:-1], means_class[['fraudulent firm']].T).T@np.subtract(fraudulent_firm.iloc[:,:-1], means_class[['fraudulent firm']].T)
covariance_not_fraudulent_firm = np.subtract(Not_fraudulent_firm.iloc[:,:-1], means_class[['Not fraudulent firm']].T).T@np.subtract(Not_fraudulent_firm.iloc[:,:-1], means_class[['Not fraudulent firm']].T)
Covarianc_matrix = (covariance_fraudulent_firm+covariance_not_fraudulent_firm)/(len(X_matrix)-X_matrix.shape[1])

In [13]:
# Linear discriminant function
Discriminant_score = X_matrix@linalg.pinv(Covarianc_matrix)@means_class.to_numpy()-np.diag((1/2)*means_class.to_numpy().T@means_class)+np.log(prior_prob).to_numpy()

In [14]:
# Fitted values
Y_fitted = pd.DataFrame(np.where(Discriminant_score.iloc[:,0]>Discriminant_score.iloc[:,1], 1, 0))

In [15]:
# Calculate error rate
Error_rate = np.sum(np.abs(Y_fitted.iloc[:,0]-Y_var.iloc[:,0]))/len(Y_fitted)
Error_rate

0.04896907216494845

###  Quadratic Discriminant Analysis

Unlike LDA, QDA assumes that each class has it's own covariance matrix. That is, it assumes that an observation from the $k^{th}$ class is of the form $X \sim N(\mu_k, \Sigma_k)$. Under this assumtion our discriminant function is of the following form:

\begin{equation}
\delta_k(x)=-\frac{1}{2}x^T\Sigma_k^{-1}x+x^T\Sigma_k^{-1}\mu_k-\frac{1}{2}(\mu_k)^T\Sigma_k^{-1}(\mu_k) -\frac{1}{2}log|\Sigma_k| +log\pi_k
\end{equation}

In [16]:
## Estimation of the paramaters of Normal distributions
means_class_fraudulent_firm = np.mean(fraudulent_firm.iloc[:,:-1])
means_class_Not_fraudulent_firm = np.mean(Not_fraudulent_firm.iloc[:,:-1])

In [17]:
Covarianc_matrix = [covariance_fraudulent_firm, covariance_not_fraudulent_firm]
means_class = [means_class_fraudulent_firm,means_class_Not_fraudulent_firm]

In [18]:
# Linear discriminant function for two classes

discriminant_score_QDA = [0]*2
for k in range(2):
    squared_term = (1/2)*X_matrix@linalg.pinv(Covarianc_matrix[k])@X_matrix.T
    linear_term = X_matrix@linalg.pinv(Covarianc_matrix[k])@means_class[k].to_numpy()
    third_term = (1/2)*means_class[k]@linalg.pinv(Covarianc_matrix[k])@means_class[k].to_numpy()
    last_terms = -(1/2)*linalg.det(Covarianc_matrix[k])+prior_prob.iloc[:,k]
    discriminant_score_QDA[k] = np.diag(-squared_term+linear_term-third_term+last_terms[0])

discriminant_score_QDA= pd.DataFrame(discriminant_score_QDA).T

In [19]:
# Fitted values
Y_fitted_QDA = pd.DataFrame(np.where(discriminant_score_QDA.iloc[:,0]>discriminant_score_QDA.iloc[:,1], 1, 0))

In [20]:
# Calculate error rate
Error_rate_QDA = np.sum(np.abs(Y_fitted_QDA.iloc[:,0]-Y_var.iloc[:,0]))/len(Y_fitted_QDA)
Error_rate_QDA

0.045103092783505154

### 4.3.1:  Regularized Discriminant Analysis

The regularized covariance matrices have the form:

\begin{equation}
\hat{\Sigma_k}(\alpha) = \alpha\hat{\Sigma_k} + (1-\alpha)\hat{\Sigma}
\end{equation}

where $\hat{\Sigma}$ is the pooled covariance matrix used in LDA.

In [21]:
# Regularized Discriminant Analysis
Reg_LDA = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
# model evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
Reg_LDA_scores = cross_val_score(Reg_LDA, X_matrix, Y, scoring='accuracy', cv=cv, n_jobs=-1)
Reg_LDA_scores.mean()

0.9394105894105894

### 4.4:  Logistic regression

#### 4.4.1:  Simple Logistic regression

The logistic regression model has the form:
\begin{equation}
log(\frac{Pr(G=K-1|X=x)}{Pr(G=K|X=x)}) = \beta_{(K-1)0}+\beta_{K-1}^Tx
\end{equation}

In [22]:
# Logistic regression
Log_reg = LogisticRegression(penalty='none', random_state=1)

In [23]:
# model evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
Log_reg_score = cross_val_score(Log_reg, X_matrix, Y, scoring='accuracy', cv=cv, n_jobs=-1)
Log_reg_score.mean()

0.9897047397047399

#### 4.4.2:  Regularized Logistic regression

The $L_1z$ penalty can be used for variable selection and shrinkage with any linear regression model including logistic regression:

\begin{equation}
max_{\beta_0, \beta}=\{ \sum_{i=1}^{N} [y_i(\beta_0+\beta^Tx_i)-log(1+e^{\beta_0+\beta^Tx_i})]-\lambda \sum_{j=1}^{p} |\beta_j| \}
\end{equation}

In [24]:
# L-1 Logistic regression
Log_reg_l1 = LogisticRegression(penalty='l2', solver = "sag", random_state=1)

In [25]:
# model evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
Log_reg_l1_score = cross_val_score(Log_reg_l1, X_matrix, Y, scoring='accuracy', cv=cv, n_jobs=-1)
Log_reg_l1_score.mean()

0.9746198246198249